# AAI-511 Final Project

Paul Parks

## Libraries

pretty-midi is used for midi calculations (key, tempo, etc):
https://github.com/craffel/pretty-midi

In [10]:
# %pip install pretty_midi 

In [11]:
import os
import pretty_midi
import numpy as np
import mido
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, Attention
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

## Data Collection
Data is collected and provided to you.

**Dataset**

The project will use a dataset consisting of musical scores from various composers. Download the dataset from Kaggle websiteLinks to an external site..

https://www.kaggle.com/datasets/blanderbuss/midi-classic-music/data

The dataset contains the midi files of compositions from well-known classical composers like Bach, Beethoven, Chopin, and Mozart. The dataset should be labeled with the name of the composer for each score. Please only do your prediction only for below composers, therefore you need to select the required composers from the given dataset above.

1. Bach
1. Beethoven
1. Chopin
1. Mozart

## Data Pre-processing
Convert the musical scores into a format suitable for deep learning models. This involves converting the musical scores into MIDI files and applying data augmentation techniques.

In [12]:
def load_midi_files(base_dir, composers):
    midi_files = []
    labels = []

    for composer in composers:
        composer_dir = os.path.join(base_dir, composer)
        for root, _, files in os.walk(composer_dir):
            for file in files:
                if file.endswith('.mid') or file.endswith('.midi'):
                    midi_files.append(os.path.join(root, file))
                    labels.append(composer)
    return midi_files, labels

base_dir = './Dataset/midiclassics/'
composers = ['Bach', 'Beethoven', 'Chopin', 'Mozart']

midi_files, labels = load_midi_files(base_dir, composers)

print('Number of MIDI files:', len(midi_files))
print('Number of labels:', len(labels))

# print files per composer
for composer in composers:
    print(f'{composer}: {labels.count(composer)}')

Number of MIDI files: 1530
Number of labels: 1530
Bach: 925
Beethoven: 212
Chopin: 136
Mozart: 257


## Feature Extraction
Extract features from the MIDI files, such as notes, chords, and tempo, using music analysis tools.


In [13]:
def pad_or_truncate(array, max_length):
    array = np.array(array)
    array = array[array > 0]
    if len(array) > max_length:
        return array[:max_length]
    else:
        return np.pad(array, (0, max_length - len(array)), 'constant')

def extract_chords(midi_data, time_window=0.05):
    chords = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            notes = sorted(instrument.notes, key=lambda note: note.start)
            current_chord = []
            current_start_time = notes[0].start if notes else None
            
            for note in notes:
                if current_start_time is not None and note.start - current_start_time > time_window:
                    if len(current_chord) > 1:
                        chords.append(current_chord)
                    current_chord = []
                    current_start_time = note.start
                
                current_chord.append(note)
            
            if len(current_chord) > 1:
                chords.append(current_chord)
    
    return chords

COMMON_CHORDS = {
    'major': (0, 4, 7),
    'minor': (0, 3, 7),
    'diminished': (0, 3, 6),
    'augmented': (0, 4, 8),
    'dominant_seventh': (0, 4, 7, 10),
    'major_seventh': (0, 4, 7, 11),
    'minor_seventh': (0, 3, 7, 10),
    'suspended_fourth': (0, 5, 7),
    'suspended_second': (0, 2, 7),
    'perfect_fourth': (0, 5, 9),
    'sixth_chord': (0, 4, 9),
    'unknown_1': (0, 4, 5, 9),
    'unknown_2': (0, 6, 9),
    'unknown_3': (0, 2, 6, 9),
    'unknown_4': (2, 5, 11),
    'unknown_5': (3, 6, 11),
    'unknown_6': (1, 6, 11),
    'other': (),
}

def extract_chord_histogram(midi_data, common_chords=COMMON_CHORDS, time_window=0.05):
    histogram = {chord: 0 for chord in common_chords}
    unknown_chords = set()
    chords = extract_chords(midi_data, time_window)
    
    for chord in chords:
        pitches = sorted(set(note.pitch % 12 for note in chord))
        if len(pitches) >= 3:
            recognized = False
            for chord_name, intervals in common_chords.items():
                if len(pitches) == len(intervals):
                    if all((pitches[i+1] - pitches[i]) % 12 == intervals[i+1] - intervals[i] for i in range(len(intervals) - 1)):
                        histogram[chord_name] += 1
                        recognized = True
                        break
            if not recognized:
                histogram['other'] += 1
                unknown_chords.add(tuple(pitches))
    
    return np.array(list(histogram.values())), unknown_chords

def extract_tempo(midi_data):
    tempos = midi_data.get_tempo_changes()
    return tempos

def extract_key_signature(midi_data):
    key_signatures = midi_data.key_signature_changes
    if key_signatures:
        key_signature = key_signatures[0].key_number
    else:
        key_signature = 0  # C major or A minor
    return key_signature

def transpose_to_c_major(midi_data):
    key_signature_changes = midi_data.key_signature_changes
    if key_signature_changes:
        original_key = key_signature_changes[0].key_number
        semitones_to_c_major = -original_key
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                note.pitch += semitones_to_c_major
    return midi_data

def extract_note_histogram(midi_data):
    histogram = np.zeros(12)
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                histogram[note.pitch % 12] += 1
    return histogram

def extract_features(midi_file):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_file)
        key_signature = extract_key_signature(midi_data)
        midi_data = transpose_to_c_major(midi_data)
        note_histogram = extract_note_histogram(midi_data)
        tempos = extract_tempo(midi_data)
        chord_histogram, unknown_chords = extract_chord_histogram(midi_data)
        pitch_classes, durations = extract_pitch_classes_and_durations(midi_data)

        return note_histogram, chord_histogram, tempos, key_signature, pitch_classes
    except (mido.KeySignatureError, KeyError) as e:
        print(f"Error processing {midi_file}: {e}")
        return None, None, None, None, None

def extract_pitch_classes_and_durations(midi_data):
    pitch_classes = []
    durations = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                pitch_classes.append(note.pitch % 12)  # Convert to pitch class
                durations.append(note.end - note.start)
    return pitch_classes, durations

def prepare_feature_data(composers):
    MAX_TEMPOS = 5

    rows = [] 
    X = []
    y = []

    for i, file in enumerate(midi_files):
        composer = labels[i]
        note_histogram, chord_histogram, tempos, key_signature, pitch_classes = extract_features(file)
        if note_histogram is not None:
            tempos = pad_or_truncate(tempos[1], MAX_TEMPOS) if len(tempos) > 1 else np.zeros(MAX_TEMPOS)

            rows.append({
                'composer': composer,
                'note_histogram': note_histogram,
                'chord_histogram': chord_histogram,
                'tempos': tempos,
                'key_signature': key_signature
            })

            
            sequence_length = 100
            # trim or pad the pitch classes to the sequence length
            pitch_classes = pad_or_truncate(pitch_classes, sequence_length)
            

            features = np.concatenate([
                note_histogram.flatten(),
                chord_histogram.flatten(),
                tempos.flatten(),
                np.array([key_signature]),
            ])
            X.append(features)

            y.append(composers.index(composer))
        else:
            print(f"Skipping {file}")

    df = pd.DataFrame(rows)

    X = pad_sequences(X, maxlen=sequence_length, padding='post')

    X_arr = np.array(X)
    y_arr = np.array(y)

    scaler = StandardScaler()
    X_arr = scaler.fit_transform(X_arr)

    X_train, X_val, y_train, y_val = train_test_split(X_arr, y_arr, test_size=0.2, random_state=42)
    return df, X, y, X_train, X_val, y_train, y_val

# Usage
df, X, y, X_train, X_val, y_train, y_val = prepare_feature_data(composers)


C:\Users\paula\AppData\Roaming\Python\Python39\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error processing ./Dataset/midiclassics/Beethoven\Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
Skipping ./Dataset/midiclassics/Beethoven\Anhang 14-3.mid
Error processing ./Dataset/midiclassics/Mozart\Piano Sonatas\Nueva carpeta\K281 Piano Sonata n03 3mov.mid: Could not decode key with 2 flats and mode 2
Skipping ./Dataset/midiclassics/Mozart\Piano Sonatas\Nueva carpeta\K281 Piano Sonata n03 3mov.mid


In [14]:
df.head()

,composer,note_histogram,chord_histogram,tempos,key_signature
0,Bach,"[136.0, 5.0, 72.0, 12.0, 65.0, 181.0, 4.0, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[120.0, 0.0, 0.0, 0.0, 0.0]",0
1,Bach,"[759.0, 195.0, 583.0, 863.0, 132.0, 806.0, 74....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[75.0, 70.00007000007, 65.000065000065, 50.0, ...",0
2,Bach,"[868.0, 82.0, 646.0, 648.0, 122.0, 691.0, 85.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[80.0, 78.00007800007802, 75.0, 70.00023333411...",0
3,Bach,"[624.0, 214.0, 860.0, 145.0, 710.0, 711.0, 147...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[60.0, 50.0, 30.0, 60.0, 55.000004583333705]",0
4,Bach,"[295.0, 243.0, 446.0, 139.0, 671.0, 70.0, 554....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[40.0, 75.0, 70.00007000007, 65.000065000065, ...",0


In [15]:
print('Number of samples:', len(X))
print('Number of labels:', len(y))
print('Feature vector length:', len(X[0]))

# count unique labels
unique_labels = set(y)
print('Unique labels:', unique_labels)

# Count samples per label
for label in unique_labels:
    print(f'{composers[label]}: {y.count(label)}')

Number of samples: 1528
Number of labels: 1528
Feature vector length: 100
Unique labels: {0, 1, 2, 3}
Bach: 925
Beethoven: 211
Chopin: 136
Mozart: 256


## Model Building
Develop a deep learning model using LSTM and CNN architectures to classify the musical scores according to the composer.


In [16]:
input_shape = (X_train.shape[1], 1)

# model = Sequential()
# model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(len(composers), activation='softmax'))
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(len(composers), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Model Training
Train the deep learning model using the pre-processed and feature-extracted data.


In [17]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model Training
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[early_stopping])


Epoch 1/100
39/39 [==============================] - 4s 41ms/step - loss: 1.2136 - accuracy: 0.6097 - val_loss: 1.2214 - val_accuracy: 0.6275
Epoch 2/100
39/39 [==============================] - 1s 21ms/step - loss: 1.1026 - accuracy: 0.6277 - val_loss: 1.1280 - val_accuracy: 0.5719
Epoch 3/100
39/39 [==============================] - 1s 20ms/step - loss: 1.0089 - accuracy: 0.6318 - val_loss: 1.0547 - val_accuracy: 0.5882
Epoch 4/100
39/39 [==============================] - 1s 20ms/step - loss: 0.9213 - accuracy: 0.6530 - val_loss: 1.0425 - val_accuracy: 0.5719
Epoch 5/100
39/39 [==============================] - 1s 20ms/step - loss: 0.9116 - accuracy: 0.6604 - val_loss: 1.0525 - val_accuracy: 0.5719
Epoch 6/100
39/39 [==============================] - 1s 20ms/step - loss: 0.8907 - accuracy: 0.6448 - val_loss: 0.9976 - val_accuracy: 0.5752
Epoch 7/100
39/39 [==============================] - 1s 20ms/step - loss: 0.8780 - accuracy: 0.6588 - val_loss: 0.9592 - val_accuracy: 0.6078
Epoch 

## Model Evaluation
Evaluate the performance of the deep learning model using accuracy, precision, and recall metrics.


In [18]:
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Accuracy: {val_accuracy}')

# Calculate additional metrics
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_val, y_pred_classes, target_names=composers))
conf_matrix = confusion_matrix(y_val, y_pred_classes)
print(conf_matrix)

Validation Accuracy: 0.6601307392120361
10/10 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

        Bach       0.73      0.93      0.81       175
   Beethoven       0.47      0.33      0.39        57
      Chopin       0.50      0.46      0.48        28
      Mozart       0.47      0.17      0.25        46

    accuracy                           0.66       306
   macro avg       0.54      0.47      0.49       306
weighted avg       0.62      0.66      0.62       306

[[162   4   2   7]
 [ 27  19   9   2]
 [ 10   5  13   0]
 [ 24  12   2   8]]


## Model Optimization
Optimize the deep learning model by fine-tuning hyperparameters.
